In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [7]:
# Build the vocabulary of characters and mappints to / from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(vocab_size)

27


In [9]:
# Build DataSet
block_size = 3 # context length: how many characters do we take to predict the next one?
def build_dataset(words):
    X, Y = [], []
    for w in words[:]:
      context = [0] * block_size
      for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix] # crop and append
      
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 =int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [11]:
n_emb = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator = g)
W1 = torch.randn((n_emb * block_size, n_hidden), generator = g) * ((5/3 ) / (n_emb * block_size ** 0.5))
b1 = torch.randn((n_hidden), generator = g) * 0.01
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.01
b2 = torch.randn((vocab_size), generator = g) * 0

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

parameters = [C, W1, W2, b1, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

12297


In [13]:
(torch.randn(1000) * 0.2).std()

tensor(0.2050)

In [15]:
hpreact.shape

NameError: name 'hpreact' is not defined

In [17]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
# Batch normalization alternatives -> Group normalization. 
  hpreact = embcat @ W1 + b1
  hpreact = bngain * ((hpreact - hpreact.mean(0, keepdim = True)) / hpreact.std(0, keepdim = True)) + bnbias
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function
  
  # backward pass 
  for p in parameters:
    p.grad = None
  loss.backward()
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters: 
    p.data += -(lr * p.grad)

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  # break

      0/ 200000: 3.3147
  10000/ 200000: 2.0977
  20000/ 200000: 2.3415
  30000/ 200000: 2.3992
  40000/ 200000: 2.0277
  50000/ 200000: 2.3060
  60000/ 200000: 2.4402
  70000/ 200000: 2.0833
  80000/ 200000: 2.3971
  90000/ 200000: 2.1310
 100000/ 200000: 1.7951
 110000/ 200000: 2.4099
 120000/ 200000: 1.9640
 130000/ 200000: 2.3886
 140000/ 200000: 2.3378
 150000/ 200000: 2.2366
 160000/ 200000: 1.8501
 170000/ 200000: 1.8672
 180000/ 200000: 2.0711
 190000/ 200000: 1.8476


In [18]:
# Calibrate the batch norm at the end of training

with torch.no_grad():
    # pass through the training set
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure the mean and standard deviation once
    bnmean = hpreact.mean(0, keepdim = True)
    bnstd = hpreact.std(0, keepdim = True)

In [19]:
-torch.tensor(1/27.0).log()

tensor(3.2958)

In [20]:
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': {Xtr, Ytr},
        'val': {Xdev, Ydev},
        'test': {Xte, Yte},
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * ((hpreact - bnmean) / bnstd) + bnbias
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0600664615631104
val 2.106653928756714


In [21]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim = 1)
        # Sample from the dist.
        ix = torch.multinomial(probs, num_samples = 1, generator = g).item()
        # shift the context window
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

mrhajimsadhitevmndhaylah.
zhyrstendrlegradleydhelitzell.
pplrodhis.
quanassephultz.
kasciyshubvrghhirgestin.
jostlyssngbertry.
bakyvder.
ystuf.
qhaylah.
quonnyshahmandalysssufpzokhlejusmfrcebvanvevodtelladirferrohhidgsptin.
shaulyss.
hurdrs.
malkirffrv.
khwrlla.
drthbetht.
khasirvabdityt.
dvassivveahnd.
ryephnn.
dhxavonndkylckennakatimvandridhb.
daphvarlquinashpauduisldelltva.


In [191]:
# b1 

In [99]:
class Linear:
    def __init__(self, fan_in, fan_out, bias = True):
        self.weight = torch.randn((fan_in, fan_out), generator=g)/fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm:
    def __init__(self, dim, eps=1e-5, momentum = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean)/torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        # print(self.out)
        return self.out
    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []


n_embd = 10
n_hidden = 100

C = torch.Generator().manual_seed(2147483637)

C = torch.randn((vocab_size, n_embd), generator = g)

layers = [
    Linear(n_embd * block_size, n_hidden), BatchNorm(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), BatchNorm(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size), BatchNorm(vocab_size),
]

with torch.no_grad():
    # Makes the prediction of last layer less
    layers[-1].gamma *= 0.1
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

47551


In [101]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = 0.1 if i < 10000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d} : {loss.item():.4f}')
    lossi.append(loss.log10().item())
    # break
# In principle a large linear function is just a single neural network. 

      0/ 200000 : 3.2938
  10000/ 200000 : 2.1750
  20000/ 200000 : 2.2692
  30000/ 200000 : 2.2090
  40000/ 200000 : 2.3630
  50000/ 200000 : 2.1190
  60000/ 200000 : 2.2416
  70000/ 200000 : 2.0860
  80000/ 200000 : 1.9099
  90000/ 200000 : 2.0437
 100000/ 200000 : 2.2271
 110000/ 200000 : 1.9269
 120000/ 200000 : 2.0295
 130000/ 200000 : 2.1561
 140000/ 200000 : 2.2428
 150000/ 200000 : 2.2365
 160000/ 200000 : 2.0979
 170000/ 200000 : 2.0917
 180000/ 200000 : 1.6993
 190000/ 200000 : 1.9925


In [102]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        x = emb.view(emb.shape[0], -1)
        for layer in layers:
            x = layer(x)
        probs = F.softmax(x, dim = 1)
        # Sample from the dist.
        ix = torch.multinomial(probs, num_samples = 1, generator = g).item()
        # shift the context window
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

C:\Users\DELL\AppData\Local\Temp\ipykernel_2776\4264691136.py:25: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  xvar = x.var(0, keepdim=True)


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
# Backpropagation